In [1]:
from datasets import load_dataset
dataset = load_dataset("OpenAssistant/oasst_top1_2023-08-25")
dataset

d:\Software\CodeApp\Anaconda\envs\peft\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating train split: 12947 examples [00:00, 191826.05 examples/s]
Generating test split: 690 examples [00:00, 97667.04 examples/s]


DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 12947
    })
    test: Dataset({
        features: ['text'],
        num_rows: 690
    })
})

In [5]:
print(dataset["train"][0]["text"])
print("====="*20)
print(dataset["train"][1]["text"])
print("====="*20)
print(dataset["test"][0]["text"])
print("====="*20)
print(dataset["test"][1]["text"])

<|im_start|>user
Consigliami 5 nomi per il mio cucciolo di dobberman<|im_end|>
<|im_start|>assistant
Ecco 5 nomi per il tuo cucciolo di dobermann:

- Zeus
- Apollo
- Thor
- Athena
- Odin<|im_end|>

<|im_start|>user
¿Crees que los robots sintientes deben tener los mismos derechos que los humanos?<|im_end|>
<|im_start|>assistant
Esta es una pregunta muy interesante y compleja. No tengo una opinión personal al respecto, pero puedo decirte lo que algunas personas piensan sobre este tema.😊

En general, hay dos posiciones opuestas sobre si los robots sintientes deben tener los mismos derechos que los humanos:

- Sí, son seres conscientes y merecen respeto La primera premisa sería un rotundo SÍ, los robots sintientes deben tener los mismos derechos que los humanos porque son seres conscientes, capaces de sentir, pensar y actuar de forma autónoma. Algunos argumentos a favor de esta postura son:

  - Los robots sintientes tienen una inteligencia artificial avanzada que les permite aprender, ada

- 数据格式已经好了
- 只需要告诉下模型，<|im_start|> 是tokens，应该被split
- <|im_end|>是停止符，不遇到这个模型就会永远生成不停下 
    - eos ： end of sequence

#### 加载另一个数据集 open Orca
- 好大，先不用

#### 加载model和tokenizer，并且量化，加载时候加上量化的参数就好了
- bitsandbytes暂时不支持Windows，要安装特定windows版本的
    - 装完要切换下vscode的环境才能激活
    - bitsandbytes-windows
    - pip install https://github.com/jllllll/bitsandbytes-windows-webui/releases/download/wheels/bitsandbytes-0.39.1-py3-none-win_amd64.whl
- **Mistral的tokenizer的这些方法的api解释在哪看？**
    - 这个模型在huggingface实现的api详解https://huggingface.co/docs/transformers/v4.36.1/en/model_doc/mistral
    - 有些`tokenizer.pad_token`这个方法没写，可能是继承了，在别的model的tokenizer里有，比如GPT-2就都有

In [3]:
    
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer, BitsAndBytesConfig
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model

modelpath="D:\Changling\Model\Mistral-7b-v0.1"
# 加载4-bit quantized model
model = AutoModelForCausalLM.from_pretrained(
    modelpath,    
    device_map="auto",
    quantization_config=BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_quant_type="nf4", 
    ),
    torch_dtype=torch.bfloat16, # 指定为bf16格式
)

# Load (slow) Tokenizer, fast tokenizer sometimes ignores added tokens
# 加载  (slow)tokenizer , fast tokenizer有时候会忽略增加的tokens
# use_fast=False即可
tokenizer = AutoTokenizer.from_pretrained(modelpath, use_fast=False)   

# Add tokens <|im_start|> and <|im_end|>, latter is special eos token 
# 增加tokens
tokenizer.pad_token = "</s>"
tokenizer.add_tokens(["<|im_start|>"])
tokenizer.add_special_tokens(dict(eos_token="<|im_end|>"))
# 重新设置模型的embedding的维度大小-》改解构也有这么方便的api、、、
model.resize_token_embeddings(len(tokenizer))
# 获取特殊token对应的id（token化后的数字）也这么方便
model.config.eos_token_id = tokenizer.eos_token_id

# 这里用了6.8GB显存左右 (初始GPU显存占用为0.8GB)


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin c:\Users\win10\.conda\envs\lins_torch113_py310\lib\site-packages\bitsandbytes\libbitsandbytes_cuda116.dll
CUDA SETUP: CUDA runtime path found: C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v11.6\bin\cudart64_110.dll
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 116
CUDA SETUP: Loading binary c:\Users\win10\.conda\envs\lins_torch113_py310\lib\site-packages\bitsandbytes\libbitsandbytes_cuda116.dll...


c:\Users\win10\.conda\envs\lins_torch113_py310\lib\site-packages\bitsandbytes\cuda_setup\main.py:156: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {WindowsPath('C:/Users/win10/.conda/envs/lins_torch113_py310/bin')}
  warn(msg)
c:\Users\win10\.conda\envs\lins_torch113_py310\lib\site-packages\bitsandbytes\cuda_setup\main.py:156: UserWarning: C:\Users\win10\.conda\envs\lins_torch113_py310 did not contain ['cudart64_110.dll', 'cudart64_120.dll', 'cudart64_12.dll'] as expected! Searching further paths...
  warn(msg)
Loading checkpoint shards: 100%|██████████| 2/2 [01:42<00:00, 51.03s/it]


In [4]:
model
# 31个block，

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32002, 4096)
    (layers): ModuleList(
      (0): MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
      (1): Mistra


#### 给模型添加LoRA adapters，在这之前需要**修改模型**适配任务的地方，**要提前修改**，不然训练的权重无法合并。
- （训练的适配器权重就不包括额外的两个token？？，那模型也不变不就可以了？
- 选择lora要训练和保存的部分？设置一下loraconfig

In [5]:
# Add LoRA adapters to model
# 在训练前统一做一些事情，1- Cast the layernorm in fp32 2- making output embedding layer require grads 3- Add the upcasting of the lm head to fp32
model = prepare_model_for_kbit_training(model)
config = LoraConfig(
    r=64, 
    lora_alpha=16, 
    target_modules = ['q_proj', 'k_proj', 'down_proj', 'v_proj', 'gate_proj', 'o_proj', 'up_proj'],
    lora_dropout=0.1, 
    bias="none", 
    modules_to_save = ["lm_head", "embed_tokens"],        # needed because we added new tokens to tokenizer/model # 指定除lora部分，还要训练的模块，并保存在最终checkpoint
    task_type="CAUSAL_LM"
)
# Returns a Peft model object from a model and a config
# 返回一个高效参数微调的model
model = get_peft_model(model, config)
model.config.use_cache = False

# 这里用了9.8G显存

In [6]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): ModulesToSaveWrapper(
          (original_module): Embedding(32002, 4096)
          (modules_to_save): ModuleDict(
            (default): Embedding(32002, 4096)
          )
        )
        (layers): ModuleList(
          (0): MistralDecoderLayer(
            (self_attn): MistralAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=4096, bias=False)
                )
                (lora_embedding_A): Paramet

#### 设置上下文截断长度
- 报错 `NameError: name 'tokenizer' is not defined` -》只用一个线程ok，num_proc=1
    - 是因为windows下使用多线程问题，上面的声明只是在一个线程中，其他线程没有。
    - 你改为传入,但测试后还是不行

- `tokenizer`也是模型，按照tokenizer中配置的规则
  - dataset.map()中的batch设置为true表示
    - 对自然语言转化为token每次加载1000（默认）个样本为一个batch（[train]里的1000个样本）进行tokenizer
- `dataset.map`之后每个样本都经过tokenizer了,保存在`dataset_tokenized`中
  - `dataset_tokenized`为字典格式，两个key
    - `input_ids`
    - `attention_masks`
  -  这怎么直接这样了？后面的collate还干什么，他微调以什么为label的？？

In [6]:
import os

def tokenize(element):
    print("element-text: "+element["text"])
    return tokenizer(
        element['text'], # 数据集的每个样本是用text作为key来索引的
        truncation=True, # 是否截断
        max_length=512, # 最大长度2048个token
        add_special_tokens=False,
    )

# dataset.map用于数据预处理，因为他可以对数据集中的每个元素应用指定的函数。
# 这里是对每个样本进行tokenize，就是将text转为数字

# dataset.map的返回值直接就是DatasetDict类型的对象，他包含一个或多个Dataset对象，一个对象就像是一个train、test测试集。直接用["train"]来访问就得到了
dataset_tokenized = dataset.map(
    tokenize,
    batched=True,
    num_proc=1, # multithreaded 多线程 ->windows下python的多线程还是垃圾
    remove_columns=["text"] # 现在不用text了，现在开始就用tokens了   dont need the strings anymore,we have tokens from here on
)

Map:   0%|          | 0/12947 [00:00<?, ? examples/s]


TypeError: can only concatenate str (not "list") to str

#### Batching 批处理：转为batch  ,  用collate函数
- `input_ids` (tokenized text) token化之后的text
- `labels` (target text,same as `input_ids`)
- `attention_masks` (tensor of zeros and ones)一堆0/1的张量
- 

- 这里用QLoRA库中`DatacollatorForCausalLM`的简化版本（自己的写一个）
- 
- collate函数
    - 将一个字典的列表转换为一个batch字典
    - to transform list of dictionaries [{input_ids:[123,..]},{.. ] to single batch dictionary {input_ids:[..], labels:[..],attention_mask:[..]}

In [8]:
# collate function - to transform list of dictionaries [{input_ids:[123,..]},{.. ] to single batch dictionary {input_ids:[..], labels:[..],attention_mask:[..]}
def collate(elements):
    tokenlist=[e["input_ids"] for e in elements]
    tokens_maxlen=max([len(t) for t in tokenlist]) # length of longest input

    input_ids,labels,attention_masks = [],[],[]
    # 每个都做padding
    for tokens in tokenlist:
        # how many pad tokens to add for this sample
        pad_len = tokens_maxlen - len(tokens)

        # pad input_ids with pad_token, labels with ignore_index (-100) and set attention_mask 1 where content, otherwise 0
        # 将tokens列表和填充列表（由pad_token_id重复pad_len次组成） 连接起来，并添加到input_ids列表中
        input_ids.append( tokens + [tokenizer.pad_token_id]*pad_len)
        # 将tokens列表和填充列表（由-100重复pad_len组成）......
        # tranformers中loss计算时，会忽略-100的label，所以这里填充-100
        # 因果语言模型的训练，就是预测下一个token，所以这里的label就是input_ids向右移动一位
        labels.append(tokens + [-100]*pad_len)
        # 生成mask，tokens长度部分为1，填充部分为0
        attention_masks.append([1]*len(tokens) + [0]*pad_len)
    
    # 组成batch 
    batch={
        "input_ids":torch.tensor(input_ids), # 先转成tensor
        "labels":torch.tensor(labels),
        "attention_mask":torch.tensor(attention_masks)
    }

    return batch

#### 使用wandb可视化训练loss

In [9]:
# import wandb
# wandb.login()

#### 超参数
- 训练多少个epoch取决于你的数据集，理想情况下，你在eval上的损失会告诉你什么时候停止训练，以及哪个checkpoint最好
    - 但有时候eval_loss升高表示过训练集过拟合了
- learning rate
    - 使用QLoRA作者建议的默认学习率
    - 对于7B或者13B模型为0.0002
    - 对于有更多参数的模型，建议较低的学习率
        - 对于33B和65B参数的模型，学习率为0.0001
- bs和梯度累计
    - 显存不够就bs调小，累积调大，乘积为16即可
    - 8x2和16x1速度上不会差多少，但是效果上一般bs=16的更好

In [10]:
bs=8        # batch size
ga_steps=2  # gradient acc. steps 梯度累计步数 
epochs=1
steps_per_epoch=len(dataset_tokenized["train"])//(bs*ga_steps) # 每轮（每个eopch）多少步

args = TrainingArguments(
    output_dir="out",
    per_device_train_batch_size=bs,
    per_device_eval_batch_size=bs,
    evaluation_strategy="steps",
    logging_dir="logs",
    logging_steps=10,
    eval_steps=steps_per_epoch,		# eval and save once per epoch  	
    save_steps=500,                 # 每500步保存一次
    gradient_accumulation_steps=ga_steps,
    num_train_epochs=epochs,
    lr_scheduler_type="constant",
    optim="paged_adamw_32bit",
    learning_rate=0.0002,
    group_by_length=True,
    fp16=True,
    ddp_find_unused_parameters=False,
)

#### 训练

In [ ]:
trainer =Trainer(
    model=model,
    tokenizer=tokenizer,
    data_collator=collate,
    train_dataset=dataset_tokenized["train"],
    eval_dataset=dataset_tokenized["test"],
    args=args
)

trainer.train()

#### 释放显存
- 下面代码不管用，还是重启内核过一会有用。

In [17]:
del model
torch.cuda.empty_cache()

#### 合并lora权重

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import torch

base_path = "D:\Changling\Model\Mistral-7b-v0.1" # input base model
adapter_path = "out/checkpoint-809/" # input: adapters
save_to = "model/Mistral-7B-finetuned" # out: 合并模型可供推理  merged model ready for inference

base_model = AutoModelForCausalLM.from_pretrained(
    base_path,
    return_dict=True,
    torch_dtype = torch.bfloat16,
    device_map="auto"

)



tokenizer = AutoTokenizer.from_pretrained(base_path)

# Add/set tokens 添加tokens （和之前代码一样
tokenizer.pad_token = "</s>"
tokenizer.add_tokens(["|<im_start>|"])
tokenizer.add_special_tokens(dict(eos_token="<|im_end|>"))
base_model.resize_token_embeddings(len(tokenizer))
base_model.config.eos_token_id = tokenizer.eos_token_id

# 加载lora adapter 并且合并
# Instantiate a PEFT model from a pretrained model and loaded PEFT weights.
# Note that the passed `model` may be modified inplace.  注意   传递的模型可能会被就地修改？？？ may be是什么意思
model = PeftModel.from_pretrained(base_model,adapter_path)

# 合并成一个整体模型进行推理
# 还有个.unload()函数，应该是不合并，然后推理
model = model.merge_and_unload()


model.save_pretrained(save_to,safe_serialization=True,max_shard_size='4GB') # shard 碎片， 导出模型的每个分片大小最大为4GB  safe_serialization=True用huggingface自己推出的模型格式safetensors，载入更快（cpu辅助
tokenizer.save_pretrained(save_to)

c:\Users\win10\.conda\envs\lins_torch113_py310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:09<00:00,  4.59s/it]



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin c:\Users\win10\.conda\envs\lins_torch113_py310\lib\site-packages\bitsandbytes\libbitsandbytes_cuda116.dll
CUDA SETUP: CUDA runtime path found: C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v11.6\bin\cudart64_110.dll
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 116
CUDA SETUP: Loading binary c:\Users\win10\.conda\envs\lins_torch113_py310\lib\site-packages\bitsandbytes\libbitsandbytes_cuda116.dll...


c:\Users\win10\.conda\envs\lins_torch113_py310\lib\site-packages\bitsandbytes\cuda_setup\main.py:156: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {WindowsPath('C:/Users/win10/.conda/envs/lins_torch113_py310/bin')}
  warn(msg)
c:\Users\win10\.conda\envs\lins_torch113_py310\lib\site-packages\bitsandbytes\cuda_setup\main.py:156: UserWarning: C:\Users\win10\.conda\envs\lins_torch113_py310 did not contain ['cudart64_110.dll', 'cudart64_120.dll', 'cudart64_12.dll'] as expected! Searching further paths...
  warn(msg)


('model/Mistral-7B-finetuned\\tokenizer_config.json',
 'model/Mistral-7B-finetuned\\special_tokens_map.json',
 'model/Mistral-7B-finetuned\\tokenizer.model',
 'model/Mistral-7B-finetuned\\added_tokens.json',
 'model/Mistral-7B-finetuned\\tokenizer.json')

#### 加载测试集

In [8]:
from datasets import load_dataset
from transformers import AutoTokenizer
# 文件夹路径就好，应该是自动识别train test等数据集信息
# path = r"C:\Users\win10\.cache\huggingface\datasets\OpenAssistant___oasst_top1_2023-08-25"
path="./data"
dataset = load_dataset(path)
print(dataset)
# modelpath = "./model/Mistral-7B-finetuned/"
modelpath = "D:\Changling\Model\Mistral-7b-v0.1"

# Load (slow) Tokenizer, fast tokenizer sometimes ignores added tokens
# 加载  (slow)tokenizer , fast tokenizer有时候会忽略增加的tokens
# use_fast=False即可
tokenizer = AutoTokenizer.from_pretrained(modelpath, use_fast=False)   

# # Add tokens <|im_start|> and <|im_end|>, latter is special eos token 
# # 增加tokens
# tokenizer.pad_token = "</s>"
# tokenizer.add_tokens(["<|im_start|>"])
# tokenizer.add_special_tokens(dict(eos_token="<|im_end|>"))
# # 重新设置模型的embedding的维度大小-》改解构也有这么方便的api、、、
# model.resize_token_embeddings(len(tokenizer))
# # 获取特殊token对应的id（token化后的数字）也这么方便
# model.config.eos_token_id = tokenizer.eos_token_id

print(dataset["test"]["text"][0])

import os

# 这里的tokenizer是个函数，直接会对传入的元素进行操作了
def tokenize(element):
    return tokenizer(
        element['text'], # 数据集的每个样本是用text作为key来索引的
        truncation=True, # 是否截断
        max_length=512, # 最大长度2048个token
        add_special_tokens=False,
    )

# dataset.map用于数据预处理，因为他可以对数据集中的每个元素应用指定的函数。
# 这里是对每个样本进行tokenize，就是将text转为数字

# dataset.map的返回值直接就是DatasetDict类型的对象，他包含一个或多个Dataset对象，一个对象就像是一个train、test测试集。直接用["train"]来访问就得到了
dataset_tokenized = dataset.map(
    tokenize,
    batched=True,
    num_proc=1, # multithreaded 多线程 ->windows下python的多线程还是垃圾
    remove_columns=["text"] # 现在不用text了，现在开始就用tokens了   dont need the strings anymore,we have tokens from here on
)
print(dataset_tokenized)


test_dataset=dataset_tokenized["test"]
print(test_dataset[0]["input_ids"])

DatasetDict({
    test: Dataset({
        features: ['text'],
        num_rows: 690
    })
})
<|im_start|>user
Explain Calculus to a primary school student<|im_end|>
<|im_start|>assistant
Calculus is a type of math that helps us understand how things change. It's like a superpower that lets us study things that are constantly moving or growing. 

There are two important parts of calculus: differentiation and integration. Let's start with differentiation. Imagine you have a line that represents how fast something is moving at different points. Calculus helps us find out how steep or flat that line is at any given point. It's like figuring out if something is going really fast or slow, or if it's speeding up or slowing down.

Now let's talk about integration. Imagine you have a line that represents how much something is changing over time. Integration helps us find out the total amount of change over a certain period. It's like adding up all the little changes to see the big picture.

Ca

In [ ]:
# collate function - to transform list of dictionaries [{input_ids:[123,..]},{.. ] to single batch dictionary {input_ids:[..], labels:[..],attention_mask:[..]}
def collate(elements):
    tokenlist=[e["input_ids"] for e in elements]
    tokens_maxlen=max([len(t) for t in tokenlist]) # length of longest input

    input_ids,labels,attention_masks = [],[],[]
    # 每个都做padding
    for tokens in tokenlist:
        # how many pad tokens to add for this sample
        pad_len = tokens_maxlen - len(tokens)

        # pad input_ids with pad_token, labels with ignore_index (-100) and set attention_mask 1 where content, otherwise 0
        # 将tokens列表和填充列表（由pad_token_id重复pad_len次组成） 连接起来，并添加到input_ids列表中
        input_ids.append( tokens + [tokenizer.pad_token_id]*pad_len)
        # 将tokens列表和填充列表（由-100重复pad_len组成）......
        labels.append(tokens + [-100]*pad_len)
        # 生成mask，tokens长度部分为1，填充部分为0
        attention_masks.append([1]*len(tokens) + [0]*pad_len)
    
    # 组成batch 
    batch={
        "input_ids":torch.tensor(input_ids), # 先转成tensor
        "labels":torch.tensor(labels),
        "attention_mask":torch.tensor(attention_masks)
    }

    return batch

In [9]:
from transformers import AutoModelForCausalLM,AutoTokenizer
from peft import PeftModel
import torch
model_path = "./model/Mistral-7B-finetuned/"
# 合并完了之后，直接加载
model = AutoModelForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)




# 测试例子
test_sentences = ["Explain computer to a primary school student"]
encodings = tokenizer(
    test_sentences,
    truncation=True, # 是否截断
    max_length=512, # 最大长度2048个token
    add_special_tokens=False,
    return_tensors='pt' # 使返回的是pytorch张量，pt=pytorch
).to("cuda")

# 使用模型推理
model.eval()
with torch.no_grad():
    outputs = model(**encodings)
    # print(tokenizer.decode(ft_model.generate(**model_input, max_new_tokens=100, repetition_penalty=1.15)[0], skip_special_tokens=True))
    

TypeError: PeftModel.from_pretrained() missing 1 required positional argument: 'model_id'

#### 推理，测试集上

In [ ]:
from transformers import AutoModelForCausalLM,AutoTokenizer
import torch

# 加载保存好的模型和分词器
model_path = "model/Mistral-7B-finetuned/"
model = AutoModelForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# 准备测试数据
